<a href="https://colab.research.google.com/github/PawanTony/ASSIGNMENT_1/blob/main/DEPLOY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pickle

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Decision Tree classifier
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Save the trained model to a file
model_filename = 'iris_model.pkl'
pickle.dump(model, open(model_filename, 'wb'))

print(f"\nTrained model saved as '{model_filename}'")
print("\nFeature Names:", feature_names)
print("Target Names:", target_names)

Model Accuracy: 1.00

Trained model saved as 'iris_model.pkl'

Feature Names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Target Names: ['setosa' 'versicolor' 'virginica']


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import pickle
import numpy as np
import threading
import time

app = Flask(__name__)

# Load the trained model
model_filename = 'iris_model.pkl'
try:
    model = pickle.load(open(model_filename, 'rb'))
except FileNotFoundError:
    print(f"Error: Model file '{model_filename}' not found. Make sure you run the training code first.")
    model = None

# Define the prediction endpoint
@app.route('/predict', methods=['POST'])
def predict():
    if model is None:
        return jsonify({'error': 'Model not loaded'}), 500
    try:
        # Get input data from the request
        data = request.get_json()
        features = data['features']

        # Convert input to a NumPy array and reshape it
        features_array = np.array(features).reshape(1, -1)

        # Make a prediction
        prediction = model.predict(features_array)[0]

        # Get the target name for the prediction
        target_names = ['setosa', 'versicolor', 'virginica']
        predicted_class = target_names[prediction]

        return jsonify({'prediction': predicted_class})

    except Exception as e:
        return jsonify({'error': str(e)}), 400

def run_app():
    app.run(debug=False, port=5000, use_reloader=False)

# Run the Flask app in a separate thread
flask_thread = threading.Thread(target=run_app)
flask_thread.daemon = True
flask_thread.start()

# Give the Flask app a moment to start
time.sleep(2)

print("Flask app should be running on http://127.0.0.1:5000")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Flask app should be running on http://127.0.0.1:5000


In [ ]:
!pip install flask flask-ngrok

In [ ]:
import json
import requests

# The local address of your Flask app in Colab
predict_url = "http://127.0.0.1:5000/predict"

# Sample input features
input_data = {"features": [6.3, 3.3, 6.0, 2.5]}

# Convert to JSON
json_data = json.dumps(input_data)

# Set headers
headers = {'Content-Type': 'application/json'}

# Initialize response to None
response = None

# Send the POST request
try:
    response = requests.post(predict_url, headers=headers, data=json_data)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Print the JSON response
    print(response.json())

except requests.exceptions.RequestException as e:
    print(f"Error sending request: {e}")
    if response is not None:
        print(f"Response status code: {response.status_code}")
        print(f"Response text: {response.text}")

INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 19:38:20] "POST /predict HTTP/1.1" 200 -


{'prediction': 'virginica'}


In [ ]:
from google.colab import files

files.download('iris_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>